#Reading the Ingestion Data

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA toyota_project;

SELECT model, year, price, transmission, mileage, fuelType, tax, mpg, engineSize
FROM toyota_raw
LIMIT 10;


model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0
GT86,2017,15998,Manual,26919,Petrol,260,36.2,2.0
GT86,2017,18522,Manual,10456,Petrol,145,36.2,2.0
GT86,2017,18995,Manual,12340,Petrol,145,36.2,2.0
GT86,2020,27998,Manual,516,Petrol,150,33.2,2.0
GT86,2016,13990,Manual,37999,Petrol,265,36.2,2.0


# Loading Raw Data

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA toyota_project")

df_raw = spark.table("toyota_raw")

df_raw.printSchema()
df_raw.show(5)
df_raw.columns


root
 |-- model: string (nullable = true)
 |-- year: long (nullable = true)
 |-- price: long (nullable = true)
 |-- transmission: string (nullable = true)
 |-- mileage: long (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- tax: long (nullable = true)
 |-- mpg: double (nullable = true)
 |-- engineSize: double (nullable = true)

+-----+----+-----+------------+-------+--------+---+----+----------+
|model|year|price|transmission|mileage|fuelType|tax| mpg|engineSize|
+-----+----+-----+------------+-------+--------+---+----+----------+
| GT86|2016|16000|      Manual|  24089|  Petrol|265|36.2|       2.0|
| GT86|2017|15995|      Manual|  18615|  Petrol|145|36.2|       2.0|
| GT86|2015|13998|      Manual|  27469|  Petrol|265|36.2|       2.0|
| GT86|2017|18998|      Manual|  14736|  Petrol|150|36.2|       2.0|
| GT86|2017|17498|      Manual|  36284|  Petrol|145|36.2|       2.0|
+-----+----+-----+------------+-------+--------+---+----+----------+
only showing top 5 rows


['model',
 'year',
 'price',
 'transmission',
 'mileage',
 'fuelType',
 'tax',
 'mpg',
 'engineSize']

#Cleaning & feature engineering
## Cast numeric columns & drop bad rows

In [0]:
from pyspark.sql import functions as F

df = df_raw.select(
    "model", "year", "price", "transmission",
    "mileage", "fuelType", "tax", "mpg", "engineSize"
)

# Cast numerics
df = (
    df.withColumn("year", F.col("year").cast("int"))
      .withColumn("price", F.col("price").cast("double"))
      .withColumn("mileage", F.col("mileage").cast("double"))
      .withColumn("tax", F.col("tax").cast("double"))
      .withColumn("mpg", F.col("mpg").cast("double"))
      .withColumn("engineSize", F.col("engineSize").cast("double"))
)

# Drop rows where key columns are missing or invalid
df = df.dropna(subset=["year", "price", "mileage", "mpg", "engineSize"])

df.show(5)
df.printSchema()


+-----+----+-------+------------+-------+--------+-----+----+----------+
|model|year|  price|transmission|mileage|fuelType|  tax| mpg|engineSize|
+-----+----+-------+------------+-------+--------+-----+----+----------+
| GT86|2016|16000.0|      Manual|24089.0|  Petrol|265.0|36.2|       2.0|
| GT86|2017|15995.0|      Manual|18615.0|  Petrol|145.0|36.2|       2.0|
| GT86|2015|13998.0|      Manual|27469.0|  Petrol|265.0|36.2|       2.0|
| GT86|2017|18998.0|      Manual|14736.0|  Petrol|150.0|36.2|       2.0|
| GT86|2017|17498.0|      Manual|36284.0|  Petrol|145.0|36.2|       2.0|
+-----+----+-------+------------+-------+--------+-----+----+----------+
only showing top 5 rows
root
 |-- model: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- transmission: string (nullable = true)
 |-- mileage: double (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- tax: double (nullable = true)
 |-- mpg: double (nullable = true)
 |-- engin

# Cleaning & feature engineering
## Feature Engineering

In [0]:
from pyspark.sql import functions as F

# Car age
df = df.withColumn("car_age", F.lit(2020) - F.col("year"))

# Price per mile (safe divide)
df = df.withColumn(
    "price_per_mile",
    F.round(F.try_divide(F.col("price"), F.col("mileage")), 4)
)

# Fuel efficiency index (safe divide)
df = df.withColumn(
    "fuel_efficiency_index",
    F.round(F.try_divide(F.col("mpg"), F.col("engineSize")), 3)
)

# Depreciation score (safe divide)
df = df.withColumn(
    "depreciation_score",
    F.round(F.try_divide(F.col("price"), F.col("car_age")), 3)
)

# Mileage category
df = df.withColumn(
    "mileage_band",
    F.when(F.col("mileage") < 20000, "Low (<20k)")
     .when(F.col("mileage") < 60000, "Medium (20–60k)")
     .when(F.col("mileage") < 100000, "High (60–100k)")
     .otherwise("Very High (>100k)")
)

df.show(10)


+-----+----+-------+------------+-------+--------+-----+----+----------+-------+--------------+---------------------+------------------+---------------+
|model|year|  price|transmission|mileage|fuelType|  tax| mpg|engineSize|car_age|price_per_mile|fuel_efficiency_index|depreciation_score|   mileage_band|
+-----+----+-------+------------+-------+--------+-----+----+----------+-------+--------------+---------------------+------------------+---------------+
| GT86|2016|16000.0|      Manual|24089.0|  Petrol|265.0|36.2|       2.0|      4|        0.6642|                 18.1|            4000.0|Medium (20–60k)|
| GT86|2017|15995.0|      Manual|18615.0|  Petrol|145.0|36.2|       2.0|      3|        0.8593|                 18.1|          5331.667|     Low (<20k)|
| GT86|2015|13998.0|      Manual|27469.0|  Petrol|265.0|36.2|       2.0|      5|        0.5096|                 18.1|            2799.6|Medium (20–60k)|
| GT86|2017|18998.0|      Manual|14736.0|  Petrol|150.0|36.2|       2.0|      3|  

# Saving Cleaned Data 

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA toyota_project")

df.write.format("delta").mode("overwrite").saveAsTable(
    "workspace.toyota_project.toyota_clean"
)

print("Saved as workspace.toyota_project.toyota_clean")


Saved as workspace.toyota_project.toyota_clean


# Validating cleaned Data

In [0]:
df_clean = spark.table("workspace.toyota_project.toyota_clean")
df_clean.show(5)


+-----+----+-------+------------+-------+--------+-----+----+----------+-------+--------------+---------------------+------------------+---------------+
|model|year|  price|transmission|mileage|fuelType|  tax| mpg|engineSize|car_age|price_per_mile|fuel_efficiency_index|depreciation_score|   mileage_band|
+-----+----+-------+------------+-------+--------+-----+----+----------+-------+--------------+---------------------+------------------+---------------+
| GT86|2016|16000.0|      Manual|24089.0|  Petrol|265.0|36.2|       2.0|      4|        0.6642|                 18.1|            4000.0|Medium (20–60k)|
| GT86|2017|15995.0|      Manual|18615.0|  Petrol|145.0|36.2|       2.0|      3|        0.8593|                 18.1|          5331.667|     Low (<20k)|
| GT86|2015|13998.0|      Manual|27469.0|  Petrol|265.0|36.2|       2.0|      5|        0.5096|                 18.1|            2799.6|Medium (20–60k)|
| GT86|2017|18998.0|      Manual|14736.0|  Petrol|150.0|36.2|       2.0|      3|  

#Notebook Analysis 
## Summary stats

In [0]:
df_clean.describe().show()


+-------+------+-----------------+-----------------+------------+------------------+--------+-----------------+------------------+-------------------+------------------+------------------+---------------------+------------------+-----------------+
|summary| model|             year|            price|transmission|           mileage|fuelType|              tax|               mpg|         engineSize|           car_age|    price_per_mile|fuel_efficiency_index|depreciation_score|     mileage_band|
+-------+------+-----------------+-----------------+------------+------------------+--------+-----------------+------------------+-------------------+------------------+------------------+---------------------+------------------+-----------------+
|  count|  6738|             6738|             6738|        6738|              6738|    6738|             6738|              6738|               6738|              6738|              6738|                 6732|              6610|             6738|
|   mean

## Average Price by Fuel Type

In [0]:
price_fuel = df_clean.groupBy("fuelType") \
        .agg(F.round(F.avg("price"), 2).alias("avg_price")) \
        .orderBy("avg_price")

display(price_fuel)


fuelType,avg_price
Petrol,9759.54
Other,14121.16
Diesel,15697.81
Hybrid,17185.47


## Price vs Mileage Relationship

In [0]:
price_mileage = df_clean.select("price", "mileage", "mileage_band")
display(price_mileage)


price,mileage,mileage_band
16000.0,24089.0,Medium (20–60k)
15995.0,18615.0,Low (<20k)
13998.0,27469.0,Medium (20–60k)
18998.0,14736.0,Low (<20k)
17498.0,36284.0,Medium (20–60k)
15998.0,26919.0,Medium (20–60k)
18522.0,10456.0,Low (<20k)
18995.0,12340.0,Low (<20k)
27998.0,516.0,Low (<20k)
13990.0,37999.0,Medium (20–60k)


## Engine Size vs MPG


In [0]:
engine_mpg = df_clean.groupBy("engineSize") \
        .agg(F.round(F.avg("mpg"), 2).alias("avg_mpg")) \
        .orderBy("engineSize")

display(engine_mpg)


engineSize,avg_mpg
0.0,64.77
1.0,65.04
1.2,48.07
1.3,55.69
1.4,74.52
1.5,63.55
1.6,55.53
1.8,75.84
2.0,52.43
2.2,45.25


In [0]:

notebook_analysis_chart = df_clean.select(
    "price",
    "mileage",
    "model",
    "fuelType",
    "engineSize"
)

display(notebook_analysis_chart)


price,mileage,model,fuelType,engineSize
16000.0,24089.0,GT86,Petrol,2.0
15995.0,18615.0,GT86,Petrol,2.0
13998.0,27469.0,GT86,Petrol,2.0
18998.0,14736.0,GT86,Petrol,2.0
17498.0,36284.0,GT86,Petrol,2.0
15998.0,26919.0,GT86,Petrol,2.0
18522.0,10456.0,GT86,Petrol,2.0
18995.0,12340.0,GT86,Petrol,2.0
27998.0,516.0,GT86,Petrol,2.0
13990.0,37999.0,GT86,Petrol,2.0


Databricks visualization. Run in Databricks to view.